In [2]:
import H20_Nakata_model

In [4]:
import datetime

import numpy as np
import pandas as pd
import pandas.io.data as web
import quandl


def setdata(s, e, t='Adj Close'):
    start = datetime.datetime(s[0], s[1], s[2])
    end = datetime.datetime(e[0], e[1], e[2])
    N225 = web.DataReader('^N225', 'yahoo', start, end)[t]
    DJI = web.DataReader('^DJI', 'yahoo', start, end)[t]
    IXIC = web.DataReader('^IXIC', 'yahoo', start, end)[t]
    jpy = web.DataReader('DEXJPUS', 'fred')
    jpy = jpy.ix[start:end]
    usdjpy = quandl.get("BOE/XUDLJYD", start_date=start, end_date=end).assign(
                    USDJPY_roc = lambda df: df["Value"].pct_change()
                ).rename(columns={
                    "Value" : "USDJPY"
                })
    eurjpy = quandl.get("BOE/XUDLBK63", start_date=start, end_date=end).assign(
                    EURJPY_roc = lambda df: df["Value"].pct_change()
                ).rename(columns={
                    "Value" : "EURJPY"
                })
    gbpjpy = quandl.get("BOE/XUDLJYS", start_date=start, end_date=end).assign(
                    EURJPY_roc = lambda df: df["Value"].pct_change()
                ).rename(columns={
                    "Value" : "GBPJPY"
                })
    usdjpy = usdjpy["USDJPY"]
    eurjpy = eurjpy["EURJPY"]
    gbpjpy = gbpjpy["GBPJPY"]
    
    dataframe = pd.concat([N225, DJI, IXIC, usdjpy, eurjpy, gbpjpy], axis=1).dropna()
    
    datas = np.array(dataframe)
    target = datas.T[0]

    return datas, target

In [5]:
data, target = setdata((2002, 10, 1), (2006, 4, 1))
data

LimitExceededError: (Status 429) (Quandl Error QELx01) You have exceeded the anonymous user limit of 50 calls per day. To make more calls today, please register for a free Quandl account and then include your API key with your requests.

In [489]:
for i in range(len(data.T)):
    data.T[i] = (data.T[i] - np.average(data.T[i]))/np.max(abs(data.T[i] - np.average(data.T[i])))

target = (target - np.average(target))/np.max(abs(target - np.average(target)))

日経平均とその他のデータとの相関。

In [492]:
print(np.corrcoef(data.T[0], data.T[1])[0, 1])
print(np.corrcoef(data.T[0], data.T[2])[0, 1])
print(np.corrcoef(data.T[0], data.T[3])[0, 1])
print(np.corrcoef(data.T[0], data.T[4])[0, 1])
print(np.corrcoef(data.T[0], data.T[5])[0, 1])

0.815115223996
0.866370655774
0.872475524899
0.766890210222
0.722850448569


In [494]:
pred = data[-1]
pred

array([ 1.        ,  0.6937186 ,  0.7103626 ,  0.55246872,  0.73558563,
        0.2457783 ])

訓練用のデータとそのラベルの作成

In [528]:
case_A, case_B, case_C, case_D = make_input(data[:-1])
label = make_label(target)

In [529]:
y_A = label[1:]
y_B = label[5:]
y_C = label[5:]
y_D = label[5:]

テスト用のデータとそのラベルの作成

In [524]:
test_data, test_target = setdata((2007, 1, 1), (2007, 12, 1))

LimitExceededError: (Status 429) (Quandl Error QELx01) You have exceeded the anonymous user limit of 50 calls per day. To make more calls today, please register for a free Quandl account and then include your API key with your requests.

In [480]:
for i in range(len(data.T)):
    test_data.T[i] = (test_data.T[i] - np.average(test_data.T[i]))/np.max(abs(test_data.T[i] - np.average(test_data.T[i])))

test_target = (test_target - np.average(test_target))/np.max(abs(test_target - np.average(test_target)))

In [530]:
test_case_A, test_case_B, test_case_C, test_case_D = make_input(test_data[:-1])

In [531]:
test_label = make_label(test_target)
test_label_A = test_label[1:]
test_label_B = test_label[5:]
test_label_C = test_label[5:]
test_label_D = test_label[5:]

In [532]:
P_A = PredictDirection_PriceChanges(case_A, y_A, pred)
P_A.main()

(lnsigma^2, lnC)= (-10, -10) 識別誤り率= 0.4304347826086956
(lnsigma^2, lnC)= (-10, -9) 識別誤り率= 0.4304347826086956
(lnsigma^2, lnC)= (-10, -8) 識別誤り率= 0.4304347826086956
(lnsigma^2, lnC)= (-10, -7) 識別誤り率= 0.4304347826086956
(lnsigma^2, lnC)= (-10, -6) 識別誤り率= 0.4304347826086956
(lnsigma^2, lnC)= (-10, -5) 識別誤り率= 0.4304347826086956
(lnsigma^2, lnC)= (-10, -4) 識別誤り率= 0.4304347826086956
(lnsigma^2, lnC)= (-10, -3) 識別誤り率= 0.4304347826086956
(lnsigma^2, lnC)= (-10, -2) 識別誤り率= 0.4304347826086956
(lnsigma^2, lnC)= (-10, -1) 識別誤り率= 0.4304347826086956
(lnsigma^2, lnC)= (-10, 0) 識別誤り率= 0.4304347826086956
(lnsigma^2, lnC)= (-10, 1) 識別誤り率= 0.4304347826086956
(lnsigma^2, lnC)= (-10, 2) 識別誤り率= 0.4304347826086956
(lnsigma^2, lnC)= (-10, 3) 識別誤り率= 0.4304347826086956
(lnsigma^2, lnC)= (-10, 4) 識別誤り率= 0.4304347826086956
(lnsigma^2, lnC)= (-10, 5) 識別誤り率= 0.4304347826086956
(lnsigma^2, lnC)= (-10, 6) 識別誤り率= 0.4304347826086956
(lnsigma^2, lnC)= (-10, 7) 識別誤り率= 0.4304347826086956
(lnsigma^2, lnC)= (-10, 8) 識別誤り率= 0

(0.05966543077354361, 28.660778814790127, 0.6782608695652174)

In [502]:
P_A.descript(test_case_A, test_label_A)

lnσ^2= -3.15066631675
lnC= 2.89994128297
識別誤り率は 0.32173913043478264
識別成功率は 0.616822429906542
[[ 98.  66.]
 [ 16.  34.]]
グリッドサーチ最高点での識別成功率は 0.616822429906542


0.616822429906542

In [503]:
P_B = PredictDirection_PriceChanges(case_B, y_B, pred)
P_B.main()

(lnsigma^2, lnC)= (-10, -10) 識別誤り率= 0.4260869565217391
(lnsigma^2, lnC)= (-10, -9) 識別誤り率= 0.4260869565217391
(lnsigma^2, lnC)= (-10, -8) 識別誤り率= 0.4260869565217391
(lnsigma^2, lnC)= (-10, -7) 識別誤り率= 0.4260869565217391
(lnsigma^2, lnC)= (-10, -6) 識別誤り率= 0.4260869565217391
(lnsigma^2, lnC)= (-10, -5) 識別誤り率= 0.4260869565217391
(lnsigma^2, lnC)= (-10, -4) 識別誤り率= 0.4260869565217391
(lnsigma^2, lnC)= (-10, -3) 識別誤り率= 0.4260869565217391
(lnsigma^2, lnC)= (-10, -2) 識別誤り率= 0.4260869565217391
(lnsigma^2, lnC)= (-10, -1) 識別誤り率= 0.4260869565217391
(lnsigma^2, lnC)= (-10, 0) 識別誤り率= 0.4260869565217391
(lnsigma^2, lnC)= (-10, 1) 識別誤り率= 0.4260869565217391
(lnsigma^2, lnC)= (-10, 2) 識別誤り率= 0.4260869565217391
(lnsigma^2, lnC)= (-10, 3) 識別誤り率= 0.4260869565217391
(lnsigma^2, lnC)= (-10, 4) 識別誤り率= 0.4260869565217391
(lnsigma^2, lnC)= (-10, 5) 識別誤り率= 0.4260869565217391
(lnsigma^2, lnC)= (-10, 6) 識別誤り率= 0.4260869565217391
(lnsigma^2, lnC)= (-10, 7) 識別誤り率= 0.4260869565217391
(lnsigma^2, lnC)= (-10, 8) 識別誤り率= 0

(0.060844383769475784, 71.965867769315636, 0.5782608695652174)

In [504]:
P_B.descript(test_case_B, test_label_B)

lnσ^2= -2.79943576012
lnC= 4.27619194786
識別誤り率は 0.4217391304347826
識別成功率は 0.49047619047619045
[[ 72.  67.]
 [ 40.  31.]]
グリッドサーチ最高点での識別成功率は 0.46190476190476193


0.46190476190476193

In [517]:
min(test_case_B.T[0])

-0.23175757245560213

In [533]:
P_C = PredictDirection_PriceChanges(case_C, y_C, pred)
P_C.main()

(lnsigma^2, lnC)= (-10, -10) 識別誤り率= 0.4260869565217391
(lnsigma^2, lnC)= (-10, -9) 識別誤り率= 0.4260869565217391
(lnsigma^2, lnC)= (-10, -8) 識別誤り率= 0.4260869565217391
(lnsigma^2, lnC)= (-10, -7) 識別誤り率= 0.4260869565217391
(lnsigma^2, lnC)= (-10, -6) 識別誤り率= 0.4260869565217391
(lnsigma^2, lnC)= (-10, -5) 識別誤り率= 0.4260869565217391
(lnsigma^2, lnC)= (-10, -4) 識別誤り率= 0.4260869565217391
(lnsigma^2, lnC)= (-10, -3) 識別誤り率= 0.4260869565217391
(lnsigma^2, lnC)= (-10, -2) 識別誤り率= 0.4260869565217391
(lnsigma^2, lnC)= (-10, -1) 識別誤り率= 0.4260869565217391
(lnsigma^2, lnC)= (-10, 0) 識別誤り率= 0.4260869565217391
(lnsigma^2, lnC)= (-10, 1) 識別誤り率= 0.4260869565217391
(lnsigma^2, lnC)= (-10, 2) 識別誤り率= 0.4260869565217391
(lnsigma^2, lnC)= (-10, 3) 識別誤り率= 0.4260869565217391
(lnsigma^2, lnC)= (-10, 4) 識別誤り率= 0.4260869565217391
(lnsigma^2, lnC)= (-10, 5) 識別誤り率= 0.4260869565217391
(lnsigma^2, lnC)= (-10, 6) 識別誤り率= 0.4260869565217391
(lnsigma^2, lnC)= (-10, 7) 識別誤り率= 0.4260869565217391
(lnsigma^2, lnC)= (-10, 8) 識別誤り率= 0

(0.15506729047053303, 379.85697721040304, 0.6217391304347826)

In [534]:
P_C.descript(test_case_C, test_label_C)

lnσ^2= -1.8638961242
lnC= 5.93979480611
識別誤り率は 0.37826086956521743
識別成功率は 0.6142857142857143
[[ 89.  58.]
 [ 23.  40.]]
グリッドサーチ最高点での識別成功率は 0.6190476190476191


0.6190476190476191

In [535]:
P_D = PredictDirection_PriceChanges(case_D, y_D, pred)
P_D.main()

(lnsigma^2, lnC)= (-10, -10) 識別誤り率= 0.4260869565217391
(lnsigma^2, lnC)= (-10, -9) 識別誤り率= 0.4260869565217391
(lnsigma^2, lnC)= (-10, -8) 識別誤り率= 0.4260869565217391
(lnsigma^2, lnC)= (-10, -7) 識別誤り率= 0.4260869565217391
(lnsigma^2, lnC)= (-10, -6) 識別誤り率= 0.4260869565217391
(lnsigma^2, lnC)= (-10, -5) 識別誤り率= 0.4260869565217391
(lnsigma^2, lnC)= (-10, -4) 識別誤り率= 0.4260869565217391
(lnsigma^2, lnC)= (-10, -3) 識別誤り率= 0.4260869565217391
(lnsigma^2, lnC)= (-10, -2) 識別誤り率= 0.4260869565217391
(lnsigma^2, lnC)= (-10, -1) 識別誤り率= 0.4260869565217391
(lnsigma^2, lnC)= (-10, 0) 識別誤り率= 0.4260869565217391
(lnsigma^2, lnC)= (-10, 1) 識別誤り率= 0.4260869565217391
(lnsigma^2, lnC)= (-10, 2) 識別誤り率= 0.4260869565217391
(lnsigma^2, lnC)= (-10, 3) 識別誤り率= 0.4260869565217391
(lnsigma^2, lnC)= (-10, 4) 識別誤り率= 0.4260869565217391
(lnsigma^2, lnC)= (-10, 5) 識別誤り率= 0.4260869565217391
(lnsigma^2, lnC)= (-10, 6) 識別誤り率= 0.4260869565217391
(lnsigma^2, lnC)= (-10, 7) 識別誤り率= 0.4260869565217391
(lnsigma^2, lnC)= (-10, 8) 識別誤り率= 0

(0.051497786397205404, 0.41626924721485847, 0.591304347826087)

In [536]:
P_D.descript(test_case_D, test_label_D)

lnσ^2= -2.96621645481
lnC= -0.876422999201
識別誤り率は 0.408695652173913
識別成功率は 0.5571428571428572
[[ 107.   88.]
 [   5.   10.]]
グリッドサーチ最高点での識別成功率は 0.5666666666666667


0.5666666666666667